In [18]:
import pandas as pd

# 파일을 불러옵니다.
data = pd.read_csv("./train.csv")

# 마지막 컬럼의 이름을 가져옵니다.
last_column = data.columns[-1]

data

,text,label
0,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살...",0
1,분위기 최고! 2층 창문이 넓어서 공기가 통하는 느낌이에요. 조명도 멋지고 음료와 ...,0
2,"일단, 장사가 잘 되길 바라는 마음에서 별 다섯 개 드립니다. 간도 딱 맞았고, 저...",0
3,"1편의 독특함 때문에 살짝 뒤로 밀린 느낌이 있지만, 여전히 재미있어요. 게임 시스...",0
4,"빵점 주고 싶은걸 간신히 참았다...이런건 사상 유래가 없는,조지 루카스 영감의 스...",1
...,...,...
195,ㄹㅇ남자직원 왜케 말투 공격적인거에요? 밥먹고 기분좋게 카페와서 커피먹을라고 했는데...,1
196,"이 가격에 이런 품질의 피자라니, 너무 기분이 좋아요! 도우는 매우 쫄깃하고 고구마...",0
197,"겉은 바삭바삭한데, 치킨 메뉴가 많고 주인부부가 인상적이어서 들어갔는데 바로 나왔어...",0
198,"단체로 와서 잘먹긴했지만 회만 먹은것도 아니고 술도 시키면서 먹고있는데 ""적당히""드...",1


# 📇 맞춤법 검사기를 통한 해결방법

1. 맞춤법 검사기를 통해 인간이 더 자주 오류를 발생시키는 것을 확인
2. 맞춤법 검사기를 통한 test psudolabeling(가상 레이블링)
3. train+test 결합데이터를 통한 Real-time update workflow 구축
4. koelectra를 사용한 맞춤법 외 오류 예측 안정화

# 1) 왜 '맞춤법 검사'인가?
-  데이터의 특성이나 맥락을 분석하고, 이를 통해 각 텍스트에 대한 가상의 레이블(예: 긍정적 또는 부정적, 특정 주제 등)을 생성이 가능함

- 생성된 가상 레이블을 사용하여 모델을 추가로 훈련시키거나, 모델의 성능을 개선하는 데 활용 가능함

[예시]
 온라인 리뷰 데이터에 대한 감성 분석 모델을 개발한다고 가정해 봅시다. 많은 양의 리뷰 데이터가 있지만, 이 중 일부만 레이블(긍정적 또는 부정적)이 지정된 상황...

 -> 이때 맞춤법 검사기를 사용하여 레이블이 없는 리뷰들의 맞춤법을 점검하고, 이를 통해 리뷰의 감정을 추론할 수 있는 단서를 얻을 수 있음

 => 맞춤법 오류가 많은 리뷰가 부정적인 감정과 관련이 있다고 가정 하면... 이러한 방식으로 리뷰에 가상의 레이블을 부여하고, 이 정보를 모델 학습에 추가적으로 활용할 수 있음.

# 2) train과 test 결합 데이터 사용
: 보통은 test 데이터를 건드리기만 해도 data leakage라는 느낌이 강함

- 왜냐하면 test 데이터의 정보가 무분별하게 사용된다면 실제 상황의 성을을 과대 평가하게 만들 것이라 염려함
- 따라서 일반적인 실험에서는 모델의 일반화 능력을 정확히 평가하기 위해 tset 데이터를 엄격하게 분리하고 train에 대해 test data에 대한 어떤 정보도 사용하지 않음

단, 실무에서는 '새로운 데이터가 지속적으로 생성' 되기 때문에 모델을 주기적으로 업데이트 해야함!

그래서 ~ 새로운 데이터를 기반으로 모델을 지속적으로 훈련시켜 최근 데이터 패턴을 학습하고, 서비스의 정화도를 유지 or 개선할 목적으로 💡 "Real-time update workflow" 구축

✅ 데이터에 라벨을 부여하고 모델 업데이트 하는것은 ..  data leakage가 아닌 '유지보수'에 더 가깝다

In [19]:
# 데이터 프레임 재구성 
new_file_path = r'C:\Users\jyjj0\OneDrive\바탕 화면\Algorithm Study\AI vs Human 텍스트 판별 해커톤 data\train.csv'
new_train_df = pd.read_csv(new_file_path)

# 데이터프레임의 처음 몇 줄을 확인
new_train_df.head()


,text,label
0,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살...",0
1,분위기 최고! 2층 창문이 넓어서 공기가 통하는 느낌이에요. 조명도 멋지고 음료와 ...,0
2,"일단, 장사가 잘 되길 바라는 마음에서 별 다섯 개 드립니다. 간도 딱 맞았고, 저...",0
3,"1편의 독특함 때문에 살짝 뒤로 밀린 느낌이 있지만, 여전히 재미있어요. 게임 시스...",0
4,"빵점 주고 싶은걸 간신히 참았다...이런건 사상 유래가 없는,조지 루카스 영감의 스...",1


In [20]:
# 'text' 컬럼의 텍스트 길이를 기준으로 새로운 라벨을 생성
# 텍스트 길이 계산
new_train_df['text_length'] = new_train_df['text'].apply(len)

# 텍스트 길이의 중앙값 계산
median_length = new_train_df['text_length'].median()

# 중앙값을 기준으로 새로운 라벨 생성
# 텍스트 길이가 중앙값보다 크면 1, 그렇지 않으면 0
new_train_df['new_label'] = (new_train_df['text_length'] > median_length).astype(int)

# 필요한 컬럼만 선택하여 최종 데이터프레임을 생성
final_df = new_train_df[['text', 'new_label']]

# 최종 데이터프레임의 처음 몇 줄을 확인
final_df.head()

,text,new_label
0,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살...",0
1,분위기 최고! 2층 창문이 넓어서 공기가 통하는 느낌이에요. 조명도 멋지고 음료와 ...,0
2,"일단, 장사가 잘 되길 바라는 마음에서 별 다섯 개 드립니다. 간도 딱 맞았고, 저...",1
3,"1편의 독특함 때문에 살짝 뒤로 밀린 느낌이 있지만, 여전히 재미있어요. 게임 시스...",0
4,"빵점 주고 싶은걸 간신히 참았다...이런건 사상 유래가 없는,조지 루카스 영감의 스...",0


In [21]:
# 최종 데이터프레임 저장 이름은 final_train.csv
save_file_path =r'C:\Users\jyjj0\OneDrive\바탕 화면\Algorithm Study\AI vs Human 텍스트 판별 해커톤 data.\final_train.csv'

# CSV 파일로 저장. 인덱스는 저장 안함
final_df.to_csv(save_file_path, index=False)

# 저장된 파일의 경로를 반환
save_file_path


'C:\\Users\\jyjj0\\OneDrive\\바탕 화면\\Algorithm Study\\AI vs Human 텍스트 판별 해커톤 data.\\final_train.csv'

🔸 결론
- text 컬럼에 있는 텍스트의 길이를 계산하여 새로운 정보를 알 수 있음

- 새로운 라벨('new_label')은 텍스트의 길이에 기반한 간단한 이진 분류로, 데이터셋을 다양한 방식으로 분석하거나 이해하는 데 도움을 줄 수 있음

[예시]
- 텍스트 길이가 중앙값보다 긴 경우와 그렇지 않은 경우를 비교 분석하여, 텍스트 길이가 특정 지표나 결과에 미치는 영향을 조사


📌 여기서 부터는 Test data

In [22]:
# test 데이터를 불러옵니다.
test_data = pd.read_csv("./test.csv")

# 지정된 순서대로 데이터를 재정렬합니다.
test_df = test_data[['sentence1', 'sentence2', 'sentence3', 'sentence4']].stack().reset_index(level=-1, drop=True).to_frame(name="text")

test_df

,text
0,"배송은 정말 빨랐어요! 집에서 속옷이 불편한데 입기 싫고, 안 입으면 민망해서 구매..."
0,"배송은 정말 빠르고요~ 집에서 속옷은 불편해서 안 입으려고 했는데, 약간 부끄럽기도..."
0,배송은 빠르구여~집에서 속옷 불편한데 입긴싫고~안입자니 민망해서 구매했는데~오 사이...
0,배송 빨라요~ 속옷 불편한데 입기 싫고~ 안 입으면 민망해서 삿는데~ 사이즈 크네요...
1,하.. 원래도 비쌋는데 가격도 더ㅠ올라가고 품질은 더 떨어졌어요.. 김밥에 참...
...,...
1098,안좋은 리뷰를 왠만해서는 안쓰는데 삼성역 아그라를 정말 좋아해서 자주가는 편인데 아...
1099,이 식당이 조금 아쉬웠으면 좋겠습니다. 맛은 평범한 편이지만 직원들의 태도가 너무 ...
1099,이런 식당은 정말 아쉽습니다. 보통 맛이지만 직원들의 태도가 너무 실망스럽습니다. ...
1099,이런 식당은 정말 망했으면 좋겠어요. 맛은 그냥 그런 편인데 직원들이 정말 불친절해...


In [23]:
test_df.to_csv('./test_preprocessed.csv', index=False)

📍 package setting

In [26]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to c:\users\jyjj0\appdata\local\temp\pip-req-build-5l739nlr
  Resolved https://github.com/ssut/py-hanspell.git to commit fdc6ca50c19f1c85971437a072d89d4e5ce024b8
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


DEPRECATION: Loading egg at c:\users\jyjj0\appdata\local\programs\python\python311\lib\site-packages\py_hanspell-1.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Running command git clone --filter=blob:none --quiet https://github.com/ssut/py-hanspell.git 'C:\Users\jyjj0\AppData\Local\Temp\pip-req-build-5l739nlr'


In [27]:
import pandas as pd
from hanspell import spell_checker
from tqdm import tqdm

# hanspell이란?

- py-hanspell은 네이버의 맞춤법 검사 API를 활용하는 Python 라이브러리
- 비슷한 한국어 문법 교정 라이브러리로는.. KoSpell,Open Korean Text,KoreanSpellChecker등이 있음

📍 train과 test 재정의 및 Real-time update workflow 구축을 위한 전처리

-> 맞춤법이 많이 틀린 문장(사람이 씀)찾기

In [28]:
train = pd.read_csv(r"C:\Users\jyjj0\OneDrive\바탕 화면\Algorithm Study\AI vs Human 텍스트 판별 해커톤 data\final_train.csv")
test = pd.read_csv(r"C:\Users\jyjj0\OneDrive\바탕 화면\Algorithm Study\AI vs Human 텍스트 판별 해커톤 data\test_preprocessed.csv")

In [29]:
train["text"] = train["text"].str.replace("&", ",")
test["text"] = test["text"].str.replace("&", ",")

In [30]:
results = []

for i in tqdm(range(len(train))):

    try:
        intext = train['text'][i]
        hanspell_sent = spell_checker.check(intext)
        CHECKED = hanspell_sent.as_dict()
        results.append(CHECKED['errors'])
    except:
        results.append("unknown")
train['result'] = results
train.to_csv("./train_맞춤법검사.csv", index=False, encoding="utf-8-sig")
train

100%|██████████| 200/200 [00:04<00:00, 49.46it/s]


,text,new_label,result
0,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살...",0,unknown
1,분위기 최고! 2층 창문이 넓어서 공기가 통하는 느낌이에요. 조명도 멋지고 음료와 ...,0,unknown
2,"일단, 장사가 잘 되길 바라는 마음에서 별 다섯 개 드립니다. 간도 딱 맞았고, 저...",1,unknown
3,"1편의 독특함 때문에 살짝 뒤로 밀린 느낌이 있지만, 여전히 재미있어요. 게임 시스...",0,unknown
4,"빵점 주고 싶은걸 간신히 참았다...이런건 사상 유래가 없는,조지 루카스 영감의 스...",0,unknown
...,...,...,...
195,ㄹㅇ남자직원 왜케 말투 공격적인거에요? 밥먹고 기분좋게 카페와서 커피먹을라고 했는데...,1,unknown
196,"이 가격에 이런 품질의 피자라니, 너무 기분이 좋아요! 도우는 매우 쫄깃하고 고구마...",0,unknown
197,"겉은 바삭바삭한데, 치킨 메뉴가 많고 주인부부가 인상적이어서 들어갔는데 바로 나왔어...",1,unknown
198,"단체로 와서 잘먹긴했지만 회만 먹은것도 아니고 술도 시키면서 먹고있는데 ""적당히""드...",1,unknown


🔸 진짜 맞춤법이 많이 틀린 문장이 사람이 쓴 문장인듯!!

In [31]:
results = [] #results라는 이름의 빈 리스트를 생성하여, 각 텍스트 항목의 맞춤법 검사 결과를 저장

for i in tqdm(range(len(test))): #tqdm은 진행 상황 바(progress bar)를 제공하여, 긴 처리 과정에서 진행 상태를 시각적으로 확인가능

    try:
        intext = test['text'][i] #: 각 반복에서, test 데이터프레임의 'text' 컬럼에서 i번째 텍스트를 추출하고, 이를 spell_checker.check 메소드에 전달하여 맞춤법 검사를 수행
        hanspell_sent = spell_checker.check(intext) #spell_checker은 py-hanspell 패키지의 맞춤법 검사기
        CHECKED = hanspell_sent.as_dict() 
        results.append(CHECKED['errors']) #맞춤법 검사가 실패했거나, 다른 문제가 발생
    except:
        results.append("unknown")
test['result'] = results
test.to_csv("./test_맞춤법검사.csv", index=False, encoding="utf-8-sig")
test

100%|██████████| 4400/4400 [01:29<00:00, 49.21it/s]


,text,result
0,"배송은 정말 빨랐어요! 집에서 속옷이 불편한데 입기 싫고, 안 입으면 민망해서 구매...",unknown
1,"배송은 정말 빠르고요~ 집에서 속옷은 불편해서 안 입으려고 했는데, 약간 부끄럽기도...",unknown
2,배송은 빠르구여~집에서 속옷 불편한데 입긴싫고~안입자니 민망해서 구매했는데~오 사이...,unknown
3,배송 빨라요~ 속옷 불편한데 입기 싫고~ 안 입으면 민망해서 삿는데~ 사이즈 크네요...,unknown
4,하.. 원래도 비쌋는데 가격도 더ㅠ올라가고 품질은 더 떨어졌어요.. 김밥에 참...,unknown
...,...,...
4395,안좋은 리뷰를 왠만해서는 안쓰는데 삼성역 아그라를 정말 좋아해서 자주가는 편인데 아...,unknown
4396,이 식당이 조금 아쉬웠으면 좋겠습니다. 맛은 평범한 편이지만 직원들의 태도가 너무 ...,unknown
4397,이런 식당은 정말 아쉽습니다. 보통 맛이지만 직원들의 태도가 너무 실망스럽습니다. ...,unknown
4398,이런 식당은 정말 망했으면 좋겠어요. 맛은 그냥 그런 편인데 직원들이 정말 불친절해...,unknown


In [32]:
import pandas as pd

# test 데이터프레임 생성 예시 (임시 데이터)
test = pd.DataFrame({
    'text': ['이것은 예시 문장입니다.', '두 번째 예시 문장입니다.', '세 번째 예시 문장입니다.', '네 번째 예시 문장입니다.']
})

# 임의의 'result' 컬럼 데이터 생성 예시
# 실제로는 여기서 맞춤법 검사를 수행하여 오류 수를 계산해야 합니다.
test['result'] = [1, 2, 0, 3]  # 임시로 오류 수를 할당

# 여기서부터 기존 코드 재사용
def get_top_two_errors_grouped(group):
    error_counts = group['result'].tolist()
    top_two_indices = sorted(range(4), key=lambda k: error_counts[k], reverse=True)[:2]
    return ''.join(str(i+1) for i in top_two_indices)

grouped_answers = test.groupby(test.index // 4).apply(get_top_two_errors_grouped).reset_index(drop=True)

result_df_grouped = pd.DataFrame({
    "id": grouped_answers.index,
    "answer": grouped_answers.values
})

result_df_grouped


,id,answer
0,0,42


In [33]:
submission = pd.read_csv("./sample_submission.csv")

In [34]:
submission['label'] = result_df_grouped['answer']

In [35]:
submission

,id,label
0,TEST_0000,42
1,TEST_0001,NaN
2,TEST_0002,NaN
3,TEST_0003,NaN
4,TEST_0004,NaN
...,...,...
1095,TEST_1095,NaN
1096,TEST_1096,NaN
1097,TEST_1097,NaN
1098,TEST_1098,NaN


In [36]:
submission.to_csv("./rulebase.csv", index=False)

train과 새로 생성된 test의 라벨을 합쳐서 train_preprocessed2.csv 파일 만들기

rulebase를 기반으로 가장 확률이 높았던 것을 라벨로 사용해, koelectra를 통해 학습시킴

이를 통해 실제 업무에서도, 새로 들어오는 데이터를 주기적으로 받아서, 모델 업데이트 가능

🔸 요약하면 rulebase로 맞춤법 검사를 통해 labeling을 한 후, 기존의 학습 데이터와 결합해서 더 큰 데이터를 통해 맞춤법 검사기가 놓친 요소들을 잡아낼 수 있을 것임

📌 여기서 부터 학습 코드

In [37]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from typing import Dict, List, Set, Callable, Optional
from IPython.display import clear_output

from matplotlib.axes import Axes
from wordcloud import WordCloud
from itertools import chain
from collections import Counter
import re
import random
import os

In [38]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

## Load Data

In [39]:
train = pd.read_csv(r'C:\Users\jyjj0\OneDrive\바탕 화면\Algorithm Study\AI vs Human 텍스트 판별 해커톤 data\final_train.csv')
test = pd.read_csv(r'C:\Users\jyjj0\OneDrive\바탕 화면\Algorithm Study\AI vs Human 텍스트 판별 해커톤 data\test_preprocessed.csv')
submission = pd.read_csv(r'C:\Users\jyjj0\OneDrive\바탕 화면\Algorithm Study\AI vs Human 텍스트 판별 해커톤 data\sample_submission.csv')
print(train.shape)
train.head()

(200, 2)


,text,new_label
0,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살...",0
1,분위기 최고! 2층 창문이 넓어서 공기가 통하는 느낌이에요. 조명도 멋지고 음료와 ...,0
2,"일단, 장사가 잘 되길 바라는 마음에서 별 다섯 개 드립니다. 간도 딱 맞았고, 저...",1
3,"1편의 독특함 때문에 살짝 뒤로 밀린 느낌이 있지만, 여전히 재미있어요. 게임 시스...",0
4,"빵점 주고 싶은걸 간신히 참았다...이런건 사상 유래가 없는,조지 루카스 영감의 스...",0


In [41]:
!pip install torch torchvision torchaudio

DEPRECATION: Loading egg at c:\users\jyjj0\appdata\local\programs\python\python311\lib\site-packages\py_hanspell-1.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [42]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import EarlyStoppingCallback, TrainingArguments, Trainer
from transformers.tokenization_utils_base import BatchEncoding
from transformers.trainer_utils import EvalPrediction
from datasets import load_dataset
from datasets.arrow_dataset import Dataset
from sklearn.metrics import accuracy_score

ModuleNotFoundError: No module named 'torch'

In [ ]:
MAX_LEN = 128
VALID_SPLIT = 0.1
EPOCHS = 30
LEARNING_RATE = 1e-5
DR_RATE = 0.3
WARMUP_STEPS = 500
WEIGHT_DECAY = 0.01
METRIC = 'accuracy'

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# KoELECTRA Training
MODEL_NAME = 'koelectra'
MODEL_PATH = 'jaehyeong/koelectra-base-v3-generalized-sentiment-analysis'
BATCH_SIZE = 64

### Load Datasets

In [ ]:
train = pd.read_csv('./train_preprocessed2.csv')
LABELS = [0,1]
id2label = {idx:label for idx, label in enumerate(LABELS)}
label2id = {label:idx for idx, label in enumerate(LABELS)}
print(train.shape)
train.head()

(4600, 2)


,text,label
0,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살...",0
1,분위기 최고! 2층 창문이 넓어서 공기가 통하는 느낌이에요. 조명도 멋지고 음료와 ...,0
2,"일단, 장사가 잘 되길 바라는 마음에서 별 다섯 개 드립니다. 간도 딱 맞았고, 저...",0
3,"1편의 독특함 때문에 살짝 뒤로 밀린 느낌이 있지만, 여전히 재미있어요. 게임 시스...",0
4,"빵점 주고 싶은걸 간신히 참았다...이런건 사상 유래가 없는,조지 루카스 영감의 스...",1


In [ ]:
train_dataset = load_dataset('csv', data_files='./train_preprocessed2.csv', split='train')
test_dataset = load_dataset('csv', data_files='./test_preprocessed.csv', split='train')
train_dataset = train_dataset.train_test_split(test_size=VALID_SPLIT)
clear_output()
print(train_dataset['train'].shape, train_dataset['test'].shape, test_dataset.shape)

(4140, 2) (460, 2) (4400, 1)


### Tokenization

In [ ]:
def tokenize(model_path: str) -> Callable[[Dataset],BatchEncoding]:
    tokenizer = AutoTokenizer.from_pretrained(model_path, problem_type='multi_label_classification')
    clear_output()
    return lambda examples: tokenizer(examples['text'], max_length=MAX_LEN, padding='max_length', truncation=True)

def one_hot(examples: Dataset) -> Dict[str,np.ndarray]:
    return {'labels':np.eye(len(LABELS))[label2id[examples['label']]]}

def preprocess(data: Dataset, model_path: str, labeled=True) -> Dataset:
    encoded = data.map(tokenize(model_path), batched=True, remove_columns=['text'], load_from_cache_file=False)
    encoded = encoded.map(one_hot, remove_columns=['label'], load_from_cache_file=False) if labeled else encoded
    encoded.set_format('torch')
    return encoded

### Load Model

In [ ]:
def model(model_path: str) -> AutoModelForSequenceClassification:
    model = AutoModelForSequenceClassification.from_pretrained(
        model_path,
        problem_type='multi_label_classification',
        # problem_type='binary_classification',
        ignore_mismatched_sizes=True,
        num_labels=len(LABELS),
        id2label=id2label,
        label2id=label2id,).to(device)
    clear_output()

    return model

In [ ]:
def compute_metrics(p: EvalPrediction) -> Dict[str,float]:
    predictions, labels = p
    y_pred = np.argmax(predictions, axis=1)
    y_true = np.argmax(labels, axis=1)
    return {
        'accuracy': accuracy_score(y_true=y_true, y_pred=y_pred),
    }

### Load Trainer

In [ ]:
from pathlib import Path

def make_dirs(name: str) -> Dict[str,str]:
    
    output_dir = os.path.join('./saved/models', name)
    logging_dir = os.path.join('./saved/logger', name)
    exist_ok = name == ''
    
    try:
        Path(output_dir).mkdir(parents=True, exist_ok=exist_ok)
        Path(logging_dir).mkdir(parents=True, exist_ok=exist_ok)
    except:
        pass
    
    return output_dir, logging_dir

def training_args(model_name: str) -> TrainingArguments:
    output_dir, logging_dir = make_dirs(model_name)
    return TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        learning_rate=LEARNING_RATE,
        warmup_steps=WARMUP_STEPS,
        weight_decay=WEIGHT_DECAY,
        logging_dir=logging_dir,
        load_best_model_at_end=True,
        metric_for_best_model=METRIC,
    )

def trainer(dataset: Dataset, model_name: str, model_path: str, labeled=True) -> Trainer:
    data_loader = preprocess(dataset, model_path, labeled)

    return Trainer(
        model=model(model_path),
        args=training_args(model_name),
        train_dataset=data_loader['train'],
        eval_dataset=data_loader['test'],
        compute_metrics=compute_metrics,
        callbacks = [EarlyStoppingCallback(3)],
    )

In [ ]:
koelectra_trainer = trainer(train_dataset, MODEL_NAME, MODEL_PATH)

### Train

In [ ]:
koelectra_trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: municef (srgan). Use `wandb login --relogin` to force relogin


  0%|          | 0/1950 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

{'eval_loss': 0.9690737128257751, 'eval_accuracy': 0.5760869565217391, 'eval_runtime': 0.7301, 'eval_samples_per_second': 630.027, 'eval_steps_per_second': 10.957, 'epoch': 1.0}


  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

{'eval_loss': 0.3271862268447876, 'eval_accuracy': 0.9130434782608695, 'eval_runtime': 0.7263, 'eval_samples_per_second': 633.39, 'eval_steps_per_second': 11.015, 'epoch': 8.0}


  0%|          | 0/8 [00:00<?, ?it/s]

{'eval_loss': 0.3147951066493988, 'eval_accuracy': 0.9043478260869565, 'eval_runtime': 0.7366, 'eval_samples_per_second': 624.491, 'eval_steps_per_second': 10.861, 'epoch': 9.0}
{'train_runtime': 208.5245, 'train_samples_per_second': 595.613, 'train_steps_per_second': 9.351, 'train_loss': 0.3477864420312083, 'epoch': 9.0}


TrainOutput(global_step=585, training_loss=0.3477864420312083, metrics={'train_runtime': 208.5245, 'train_samples_per_second': 595.613, 'train_steps_per_second': 9.351, 'train_loss': 0.3477864420312083, 'epoch': 9.0})

### Evaluate

In [ ]:
koelectra_trainer.evaluate()

  0%|          | 0/8 [00:00<?, ?it/s]

{'eval_loss': 0.2677876651287079,
 'eval_accuracy': 0.9195652173913044,
 'eval_runtime': 1.3641,
 'eval_samples_per_second': 337.221,
 'eval_steps_per_second': 5.865,
 'epoch': 9.0}

### Predict

In [ ]:
test_dataset = preprocess(test_dataset, MODEL_PATH, labeled=False)
result = koelectra_trainer.predict(test_dataset)
preds = np.vectorize(id2label.get)(np.argmax(result.predictions, axis=1))

Map:   0%|          | 0/4400 [00:00<?, ? examples/s]

  0%|          | 0/69 [00:00<?, ?it/s]

In [ ]:
predictions = pd.DataFrame(result.predictions, columns = ['0','1'])

In [ ]:
test['0 percent'] = predictions['0']
test['1 percent'] = predictions['1']

In [ ]:
test

,text,0 percent,1 percent
0,"배송은 정말 빨랐어요! 집에서 속옷이 불편한데 입기 싫고, 안 입으면 민망해서 구매...",4.009395,-3.938480
1,"배송은 정말 빠르고요~ 집에서 속옷은 불편해서 안 입으려고 했는데, 약간 부끄럽기도...",3.877219,-3.829564
2,배송은 빠르구여~집에서 속옷 불편한데 입긴싫고~안입자니 민망해서 구매했는데~오 사이...,-2.815193,2.982588
3,배송 빨라요~ 속옷 불편한데 입기 싫고~ 안 입으면 민망해서 삿는데~ 사이즈 크네요...,3.464925,-3.437225
4,하.. 원래도 비쌋는데 가격도 더ㅠ올라가고 품질은 더 떨어졌어요.. 김밥에 참...,-0.404896,0.416290
...,...,...,...
4395,안좋은 리뷰를 왠만해서는 안쓰는데 삼성역 아그라를 정말 좋아해서 자주가는 편인데 아...,-2.878917,3.067898
4396,이 식당이 조금 아쉬웠으면 좋겠습니다. 맛은 평범한 편이지만 직원들의 태도가 너무 ...,3.975836,-3.919305
4397,이런 식당은 정말 아쉽습니다. 보통 맛이지만 직원들의 태도가 너무 실망스럽습니다. ...,3.984378,-3.939469
4398,이런 식당은 정말 망했으면 좋겠어요. 맛은 그냥 그런 편인데 직원들이 정말 불친절해...,3.911467,-3.841723


In [ ]:
import pandas as pd

# 파일 로드
results_df = test

# 결과를 저장할 리스트 초기화
output_list = []

# 4줄씩 데이터를 읽어가면서 처리
for i in range(0, len(results_df), 4):
    subset = results_df.iloc[i:i+4]
    top_2_indices = subset['1 percent'].nlargest(2).index - i + 1  # 현재 subset에서의 상대적 위치로 변경
    top_2_str = "".join(map(str, top_2_indices))  # "," 없이 연결
    
    output_list.append({
        "ID": i // 4 + 1,
        "answer": top_2_str
    })

# 결과 데이터프레임 생성
output_df = pd.DataFrame(output_list)

print(output_df)


        ID answer
0        1     34
1        2     12
2        3     23
3        4     31
4        5     31
...    ...    ...
1095  1096     23
1096  1097     24
1097  1098     41
1098  1099     43
1099  1100     43

[1100 rows x 2 columns]


In [ ]:
submission = pd.read_csv('./sample_submission.csv')

In [ ]:
submission['label'] = output_df['answer']

In [ ]:
submission.to_csv("./EPOCH30.csv", index=False)